### Preprocessing

In [2]:
import pandas as pd 
import numpy as np 
import os
import cv2
import matplotlib.pyplot as plt
import warnings

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow import keras
import tensorflow 

import scipy
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))


# Set the seed value for experiment reproduci.bility.
seed = 1842
tensorflow.random.set_seed(seed)
np.random.seed(seed)
# Turn off warnings for cleaner looking notebook
warnings.simplefilter('ignore')


df=pd.read_csv('data/train.csv')

X=df.drop('label',axis=1)
Y=df.label
Y=pd.get_dummies(Y,prefix='Num')

X=X/255

X=X.values.reshape(-1,28,28)

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

validation_data=X_val,y_val

Num GPUs Available:  1


In [30]:
def Charts(history,X='loss',Y='AUC',Z='Accuracy'):
    plt.plot(history.history[X])
    plt.plot(history.history[f'val_{X}'])
    plt.title(f'model {X}')
    plt.ylabel(f'{X}')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    
    plt.plot(history.history[Y])
    plt.plot(history.history[f'val_{Y}'])
    plt.title(f'model {Y}')
    plt.ylabel(f'{Y}')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

    plt.plot(history.history[Z])
    plt.plot(history.history[f'val_{Z}'])
    plt.title(f'model {Z}')
    plt.ylabel(f'{Z}')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

### Model

#### İlk CNN Score: 0.9808333516120911

In [63]:
#experiment with Convolutional neural net
model = keras.models.Sequential([

keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape = (28,28,1)),
keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
keras.layers.MaxPooling2D(),
keras.layers.BatchNormalization(),

keras.layers.Conv2D(64, (2, 2), activation='relu'),
keras.layers.Conv2D(64, (2, 2), activation='relu'),
keras.layers.Conv2D(64, (2, 2), activation='relu'),
keras.layers.MaxPooling2D(),
keras.layers.BatchNormalization(),

keras.layers.Conv2D(128, (2, 2), activation='relu'),
keras.layers.Conv2D(128, (2, 2), activation='relu'),
keras.layers.Conv2D(128, (2, 2), activation='relu'),
keras.layers.MaxPooling2D(),
keras.layers.BatchNormalization(),

keras.layers.Flatten(),
keras.layers.Dropout(0.3),
keras.layers.Dense(512, activation='relu'),
keras.layers.Dropout(0.25),
keras.layers.Dense(10, activation ='softmax')])

#### İkinci CNN

In [13]:
#experiment with Convolutional neural net
model = keras.models.Sequential([
keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape = [28,28,1]),
keras.layers.MaxPooling2D(),
keras.layers.Conv2D(32, (2, 2), activation='relu'),
keras.layers.MaxPooling2D(),

keras.layers.SeparableConv2D(64, 3, activation='relu', padding='same'),
keras.layers.SeparableConv2D(64, 3, activation='relu', padding='same'),
keras.layers.BatchNormalization(),
keras.layers.MaxPool2D(),

keras.layers.SeparableConv2D(128, 3, activation='relu', padding='same'),
keras.layers.SeparableConv2D(128, 3, activation='relu', padding='same'),
keras.layers.BatchNormalization(),
keras.layers.MaxPool2D(),
keras.layers.Dropout(0.2),

keras.layers.Flatten(),

keras.layers.Dense(512, activation='relu'),
keras.layers.BatchNormalization(),
keras.layers.Dropout(0.7),

keras.layers.Dense(128, activation='relu'),
keras.layers.BatchNormalization(),
keras.layers.Dropout(0.5),

keras.layers.Dense(64, activation='relu'),
keras.layers.BatchNormalization(),
keras.layers.Dropout(0.3),

keras.layers.Dense(10, activation ='softmax')])



#### Compile/Training 

In [64]:
model.compile(optimizer='adam',
loss=tensorflow.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc'),'acc'])
callback = keras.callbacks.EarlyStopping(monitor='val_acc',
                                            patience=7,
                                            restore_best_weights=True)


def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 20)

lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

checkpoint_cb = keras.callbacks.ModelCheckpoint("best_model.h5",
                                                    save_best_only=True)

early_stopping_cb = keras.callbacks.EarlyStopping(patience=6,restore_best_weights=True)

In [65]:
history=model.fit(X_train,y_train, epochs=40, validation_data=validation_data, callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler])

Epoch 1/40
1050/1050 [==============================] - 9s 7ms/step - loss: 0.3197 - auc: 0.9902 - acc: 0.9069 - val_loss: 0.9343 - val_auc: 0.9901 - val_acc: 0.9413 - lr: 0.0100
Epoch 2/40
1050/1050 [==============================] - 8s 7ms/step - loss: 0.1473 - auc: 0.9964 - acc: 0.9589 - val_loss: 0.1326 - val_auc: 0.9957 - val_acc: 0.9654 - lr: 0.0089
Epoch 3/40
1050/1050 [==============================] - 8s 8ms/step - loss: 0.1130 - auc: 0.9970 - acc: 0.9690 - val_loss: 0.1007 - val_auc: 0.9979 - val_acc: 0.9831 - lr: 0.0079
Epoch 4/40
1050/1050 [==============================] - 8s 8ms/step - loss: 0.0955 - auc: 0.9975 - acc: 0.9745 - val_loss: 0.0574 - val_auc: 0.9987 - val_acc: 0.9852 - lr: 0.0071
Epoch 5/40
1050/1050 [==============================] - 8s 8ms/step - loss: 0.0793 - auc: 0.9982 - acc: 0.9786 - val_loss: 0.3575 - val_auc: 0.9964 - val_acc: 0.9744 - lr: 0.0063
Epoch 6/40
1050/1050 [==============================] - 8s 8ms/step - loss: 0.0719 - auc: 0.9983 - acc: 0

In [66]:
from tensorflow.keras.models import load_model
model=load_model(
    "best_model.h5", custom_objects=None, compile=True, options=None
)
loss, auc, accuracy = model.evaluate(X_val,y_val)
print("Loss: ", loss)
print("AUC: ", auc)
print("Accuracy: ", accuracy)

263/263 [==============================] - 2s 5ms/step - loss: 0.0252 - auc: 0.9993 - acc: 0.9935
Loss:  0.025196395814418793
AUC:  0.9993237257003784
Accuracy:  0.9934523701667786


### Submission

In [67]:
df_test=pd.read_csv('test.csv')
df_test=df_test/255

df_test=df_test.values.reshape(-1,28,28)

y_pred=model.predict(df_test)


y_pred=pd.DataFrame(np.argmax(y_pred,axis=1),columns=['Label'])
submission = pd.read_csv("sample_submission.csv")
submission["Label"] = y_pred
submission.to_csv('submission.csv', index=False)

875/875 [==============================] - 2s 3ms/step


In [68]:
!kaggle competitions submit -c digit-recognizer -f submission.csv -m ""

Successfully submitted to Digit Recognizer



  0%|          | 0.00/235k [00:00<?, ?B/s]
  3%|▎         | 8.00k/235k [00:00<00:02, 79.9kB/s]
 88%|████████▊ | 208k/235k [00:00<00:00, 902kB/s]  
100%|██████████| 235k/235k [00:01<00:00, 144kB/s]
